In [1]:
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import fitz  # PyMuPDF
import pdfplumber
import torch
import numpy as np


/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# pdf extraction
def extract_text_pymupdf(path):
    doc = fitz.open(path)
    return "\n".join([page.get_text() for page in doc])

def extract_tables_pdfplumber(path):
    tables = []
    with pdfplumber.open(path) as pdf:
        for page in pdf.pages:
            tables.extend(page.extract_tables())
    return tables

pdf_path = "../data/pdfchat_sample.pdf"  # Replace with your file
raw_text = extract_text_pymupdf(pdf_path)
print(raw_text[:1000])  # Preview first 1000 characters


The History of Paper 
 
Paper has been one of the most transformative inventions in human history. Originating in 
China around 105 AD, it replaced earlier writing surfaces like papyrus and parchment. Cai 
Lun, a Chinese court official, is credited with refining the papermaking process using 
mulberry bark, hemp, and rags. 
 
Over the centuries, papermaking spread across Asia, the Middle East, and Europe. By the 
13th century, paper mills were operating in Spain and Italy, revolutionizing communication 
and record-keeping. 
 
In the modern era, paper is made primarily from wood pulp, and its uses range from books 
and newspapers to packaging and hygiene products. Despite the rise of digital media, 
paper remains a vital part of global infrastructure. 
 
Fun Fact: 
The word “paper” comes from “papyrus,” a plant-based writing material used in ancient 
Egypt. 
 
Page 2 
 
Environmental Impact 
 
While paper is recyclable and biodegradable, its production can be resource-intensive. 
Sustai

In [9]:
#chunking
def chunk_text(text, chunk_size=500):
    words = text.split()
    return [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

chunks = chunk_text(raw_text)
print(f"Total chunks: {len(chunks)}")


Total chunks: 1


In [10]:
#Embedding with BERT
bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
bert_model = AutoModel.from_pretrained("bert-base-uncased")

def embed_text(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

chunk_embeddings = [embed_text(c, bert_tokenizer, bert_model) for c in chunks]


In [11]:
#Question Embedding + Retrieval
retrieval_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
retrieval_model = AutoModel.from_pretrained("distilbert-base-uncased")

def embed_question(question):
    inputs = retrieval_tokenizer(question, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = retrieval_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

def retrieve_top_chunks(question, chunk_embeddings, chunks, k=3):
    q_embed = embed_question(question)
    scores = cosine_similarity([q_embed], chunk_embeddings)[0]
    top_indices = np.argsort(scores)[-k:][::-1]
    return [chunks[i] for i in top_indices]

question = "Where was paper first invented?"
top_chunks = retrieve_top_chunks(question, chunk_embeddings, chunks)
print("\n\n".join(top_chunks))


The History of Paper Paper has been one of the most transformative inventions in human history. Originating in China around 105 AD, it replaced earlier writing surfaces like papyrus and parchment. Cai Lun, a Chinese court official, is credited with refining the papermaking process using mulberry bark, hemp, and rags. Over the centuries, papermaking spread across Asia, the Middle East, and Europe. By the 13th century, paper mills were operating in Spain and Italy, revolutionizing communication and record-keeping. In the modern era, paper is made primarily from wood pulp, and its uses range from books and newspapers to packaging and hygiene products. Despite the rise of digital media, paper remains a vital part of global infrastructure. Fun Fact: The word “paper” comes from “papyrus,” a plant-based writing material used in ancient Egypt. Page 2 Environmental Impact While paper is recyclable and biodegradable, its production can be resource-intensive. Sustainable forestry practices and re

In [12]:
#Answer Generation with FLAN-T5
flan_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
flan_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

def generate_answer(question, context_chunks):
    context = "\n".join(context_chunks)
    prompt = f"Answer the question based on this context:\n{context}\n\nQuestion: {question}"
    inputs = flan_tokenizer(prompt, return_tensors="pt", truncation=True)
    outputs = flan_model.generate(**inputs, max_length=200)
    return flan_tokenizer.decode(outputs[0], skip_special_tokens=True)

answer = generate_answer(question, top_chunks)
print(f"\n Answer: {answer}")



 Answer: China


In [13]:
# PDF extraction using PyMuPDF
import fitz  # PyMuPDF

def extract_text_pymupdf(path):
    doc = fitz.open(path)
    return "\n".join([page.get_text() for page in doc])

# Adjust path based on notebook location
pdf_path = "../data/pdfchat_sample.pdf"  
raw_text = extract_text_pymupdf(pdf_path)

print(raw_text[:1000])  # Preview first 1000 characters


The History of Paper 
 
Paper has been one of the most transformative inventions in human history. Originating in 
China around 105 AD, it replaced earlier writing surfaces like papyrus and parchment. Cai 
Lun, a Chinese court official, is credited with refining the papermaking process using 
mulberry bark, hemp, and rags. 
 
Over the centuries, papermaking spread across Asia, the Middle East, and Europe. By the 
13th century, paper mills were operating in Spain and Italy, revolutionizing communication 
and record-keeping. 
 
In the modern era, paper is made primarily from wood pulp, and its uses range from books 
and newspapers to packaging and hygiene products. Despite the rise of digital media, 
paper remains a vital part of global infrastructure. 
 
Fun Fact: 
The word “paper” comes from “papyrus,” a plant-based writing material used in ancient 
Egypt. 
 
Page 2 
 
Environmental Impact 
 
While paper is recyclable and biodegradable, its production can be resource-intensive. 
Sustai

In [15]:
import evaluate

rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")

reference = "Paper was first invented in China."
prediction = answer  # From FLAN-T5

# ROUGE: accepts strings
rouge_scores = rouge.compute(predictions=[prediction], references=[reference])

# BLEU: accepts strings or list of strings
bleu_scores = bleu.compute(predictions=[prediction], references=[[reference]])

print("ROUGE:", rouge_scores)
print("BLEU:", bleu_scores)



ROUGE: {'rouge1': np.float64(0.2857142857142857), 'rouge2': np.float64(0.0), 'rougeL': np.float64(0.2857142857142857), 'rougeLsum': np.float64(0.2857142857142857)}
BLEU: {'bleu': 0.0, 'precisions': [1.0, 0.0, 0.0, 0.0], 'brevity_penalty': 0.0024787521766663585, 'length_ratio': 0.14285714285714285, 'translation_length': 1, 'reference_length': 7}
